# Client 2 analysis
This aims to implement a cube and do presentation
## Cube

In [1]:
import atoti as tt

Welcome to Atoti 0.8.10!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


In [2]:
session = tt.Session(
    user_content_storage=".content",
    port=9092,
    java_options=["-Xms1G", "-Xmx10G"]
)

In [3]:
db_name = "project1"
db_user = "postgres"
db_password = "postgres"  # Update with your password
db_host = "pgdb"  # Update if your DB is hosted elsewhere
db_port = "5432"

jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?user={db_user}&password={db_password}"

In [4]:
Mentalillness = session.read_sql(
    "SELECT * FROM FactHealth",
    keys=["countryid"],
    table_name="Mentalillness",
    url=jdbc_url,
)

In [5]:
dimRegiontable = session.read_sql(
    "SELECT * FROM Dimcountryarea",
    keys=["countryid"],
    table_name="Region",
    url=jdbc_url)

In [6]:
DimHealthcostGPT = session.read_sql(
    "SELECT * FROM DimHealthcost",
    keys=["healthcostid"],
    table_name="HealthcostGPT",
    url=jdbc_url
)

In [7]:
DimInternet = session.read_sql(
    "SELECT * FROM DimInternet",
    keys=["internetid"],
    table_name="Internetpercountry",
    url=jdbc_url
)

In [8]:
Dimhostee = session.read_sql(
    "SELECT * FROM Dimhostee",
    keys=["hostid"],
    table_name="hostcountry",
    url=jdbc_url
)

In [9]:
Sportdim = session.read_sql(
    "SELECT * FROM DimSport",
    keys=["medelid"],
    table_name="Sport level",
    url=jdbc_url
)

In [10]:
Mentalillness.head()

,health expenditure(GPT%),healthcostid,internetid,Average Annual Growth Rate (%),healthdifference,depressive disorders rate of Change,medelid,hostid,GDP(pc)
countryid,,,,,,,,,
13,4.36498,3,3,2.289044,-52.856722,-0.68531,3,0,3571.556907
21,11.977593,2,3,0.653723,3320.718908,-3.869335,2,1,40318.41692
33,6.32118,3,3,1.831519,54.167757,-5.84975,3,0,3354.157303
48,5.173518,3,3,0.895754,-17.11088,4.182814,3,0,1139.189892
57,4.308119,3,3,3.231297,1.52738,4.251002,3,0,822.906137


In [11]:
Mentalillness.join(dimRegiontable, Mentalillness["countryid"] == dimRegiontable["countryid"])

In [12]:
Mentalillness.join(DimHealthcostGPT, Mentalillness["healthcostid"] == DimHealthcostGPT["healthcostid"])

In [13]:
Mentalillness.join(DimInternet,
                         Mentalillness["internetid"] == DimInternet["internetid"])

In [14]:
Mentalillness.join(Dimhostee, Mentalillness["hostid"] == Dimhostee["hostid"])

In [15]:
Mentalillness.join(Sportdim, Mentalillness["medelid"] == Sportdim["medelid"])

In [16]:
session.tables.schema

```mermaid
erDiagram
  "Mentalillness" {
    _ long PK "countryid"
    nullable double "health expenditure(GPT%)"
    nullable int "healthcostid"
    nullable int "internetid"
    nullable double "Average Annual Growth Rate (%)"
    nullable double "healthdifference"
    nullable double "depressive disorders rate of Change"
    nullable int "medelid"
    nullable int "hostid"
    nullable double "GDP(pc)"
  }
  "hostcountry" {
    _ int PK "hostid"
    _ String "hostbefore"
  }
  "Sport level" {
    _ int PK "medelid"
    _ String "sportlevel"
  }
  "Region" {
    _ String "country"
    _ String "region"
    _ long PK "countryid"
  }
  "Internetpercountry" {
    _ int PK "internetid"
    _ String "Internet Level"
  }
  "HealthcostGPT" {
    _ int PK "healthcostid"
    _ String "Healthcost Level"
  }
  "Mentalillness" }o--o| "Region" : "`countryid` == `countryid`"
  "Mentalillness" }o--o| "Internetpercountry" : "`internetid` == `internetid`"
  "Mentalillness" }o--o| "HealthcostGPT" : "`healthcostid` == `healthcostid`"
  "Mentalillness" }o--o| "Sport level" : "`medelid` == `medelid`"
  "Mentalillness" }o--o| "hostcountry" : "`hostid` == `hostid`"
```


In [17]:
cube = session.create_cube(Mentalillness)

In [18]:
cube

In [19]:
hierarchies, levels, measures = cube.hierarchies, cube.levels, cube.measures

In [20]:
# original hierarchies
hierarchies

{('HealthcostGPT', 'Healthcost Level'): <atoti.hierarchy.Hierarchy object at 0x7fe28de2d9d0>, ('Region', 'country'): <atoti.hierarchy.Hierarchy object at 0x7fe28c452fd0>, ('Region', 'region'): <atoti.hierarchy.Hierarchy object at 0x7fe28c4530d0>, ('Mentalillness', 'countryid'): <atoti.hierarchy.Hierarchy object at 0x7fe28c453190>, ('hostcountry', 'hostbefore'): <atoti.hierarchy.Hierarchy object at 0x7fe28c453250>, ('Internetpercountry', 'Internet Level'): <atoti.hierarchy.Hierarchy object at 0x7fe28c453310>, ('Sport level', 'sportlevel'): <atoti.hierarchy.Hierarchy object at 0x7fe28c4533d0>}

In [21]:
# original levels
levels

In [22]:
# dimensions
# Customer: CustomerName, Gender
# SalesPerson: SalesPersonName, City, State, Country
# Product: ProductName
# Store: StoreName, City, State

In [23]:
hierarchies["Region"] = [ levels['region'],levels["country"]]

In [24]:
hierarchies

{('HealthcostGPT', 'Healthcost Level'): <atoti.hierarchy.Hierarchy object at 0x7fe28c47b4d0>, ('Region', 'country'): <atoti.hierarchy.Hierarchy object at 0x7fe28c478c50>, ('Region', 'Region'): <atoti.hierarchy.Hierarchy object at 0x7fe28c478a90>, ('Region', 'region'): <atoti.hierarchy.Hierarchy object at 0x7fe28c478410>, ('Mentalillness', 'countryid'): <atoti.hierarchy.Hierarchy object at 0x7fe28c478190>, ('hostcountry', 'hostbefore'): <atoti.hierarchy.Hierarchy object at 0x7fe28c478090>, ('Internetpercountry', 'Internet Level'): <atoti.hierarchy.Hierarchy object at 0x7fe28c460ad0>, ('Sport level', 'sportlevel'): <atoti.hierarchy.Hierarchy object at 0x7fe28c463fd0>}

In [25]:
# clean Region
del hierarchies[('Region', 'country')]
del hierarchies[('Region', 'region')]

In [26]:
# clean product
del hierarchies[('Mentalillness', 'countryid')]

In [27]:
hierarchies

{('HealthcostGPT', 'Healthcost Level'): <atoti.hierarchy.Hierarchy object at 0x7fe28de1f710>, ('Region', 'Region'): <atoti.hierarchy.Hierarchy object at 0x7fe28f0b7d90>, ('hostcountry', 'hostbefore'): <atoti.hierarchy.Hierarchy object at 0x7fe28dde5590>, ('Internetpercountry', 'Internet Level'): <atoti.hierarchy.Hierarchy object at 0x7fe28de11b10>, ('Sport level', 'sportlevel'): <atoti.hierarchy.Hierarchy object at 0x7fe28de13610>}

In [28]:
measures

{'contributors.COUNT': <atoti.measure.Measure object at 0x7fe28de37e10>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x7fe28de36d50>, 'healthdifference.SUM': <atoti.measure.Measure object at 0x7fe28de37ad0>, 'healthdifference.MEAN': <atoti.measure.Measure object at 0x7fe28de37bd0>, 'GDP(pc).SUM': <atoti.measure.Measure object at 0x7fe28c47cf90>, 'GDP(pc).MEAN': <atoti.measure.Measure object at 0x7fe28c47c610>, 'Average Annual Growth Rate (%).SUM': <atoti.measure.Measure object at 0x7fe28c47f050>, 'Average Annual Growth Rate (%).MEAN': <atoti.measure.Measure object at 0x7fe28c47cb90>, 'depressive disorders rate of Change.SUM': <atoti.measure.Measure object at 0x7fe28c47d2d0>, 'depressive disorders rate of Change.MEAN': <atoti.measure.Measure object at 0x7fe28c47c050>, 'internetid.SUM': <atoti.measure.Measure object at 0x7fe28c47d210>, 'internetid.MEAN': <atoti.measure.Measure object at 0x7fe28c47dc90>, 'medelid.SUM': <atoti.measure.Measure object at 0x7fe28c47d910>, 'medelid.MEAN': <atoti.measure.Measure object at 0x7fe28c47dd10>, 'hostid.SUM': <atoti.measure.Measure object at 0x7fe28c47d750>, 'hostid.MEAN': <atoti.measure.Measure object at 0x7fe28c47fd10>, 'health expenditure(GPT%).SUM': <atoti.measure.Measure object at 0x7fe28c47fe10>, 'health expenditure(GPT%).MEAN': <atoti.measure.Measure object at 0x7fe28c47fb50>, 'healthcostid.SUM': <atoti.measure.Measure object at 0x7fe28c47fb10>, 'healthcostid.MEAN': <atoti.measure.Measure object at 0x7fe28c47fb90>}

In [29]:
del measures["medelid.SUM"]
del measures["medelid.MEAN"]
del measures["internetid.SUM"]
del measures["internetid.MEAN"]
del measures["hostid.MEAN"]
del measures["hostid.SUM"]
del measures["healthdifference.MEAN"]
del measures["healthcostid.SUM"]
del measures["healthcostid.MEAN"]
del measures["health expenditure(GPT%).MEAN"] 
del measures["depressive disorders rate of Change.MEAN"]
del measures["GDP(pc).MEAN"]
del measures['Average Annual Growth Rate (%).MEAN']
del measures["contributors.COUNT"]

measures

{'update.TIMESTAMP': <atoti.measure.Measure object at 0x7fe28de2f3d0>, 'healthdifference.SUM': <atoti.measure.Measure object at 0x7fe28de2e9d0>, 'GDP(pc).SUM': <atoti.measure.Measure object at 0x7fe28de2e550>, 'Average Annual Growth Rate (%).SUM': <atoti.measure.Measure object at 0x7fe28de2f710>, 'depressive disorders rate of Change.SUM': <atoti.measure.Measure object at 0x7fe28de2dc50>, 'health expenditure(GPT%).SUM': <atoti.measure.Measure object at 0x7fe28de2cd90>}

## Query 1
Which countries have passed through a decline in their year-over-year population growth rate after the beginning of the 21st century?

In [30]:
cube.query(measures['Average Annual Growth Rate (%).SUM'], levels=[levels[('Region', 'Region','country')]])

Average Annual Growth Rate (%).SUM
region        country                                        
Africa        Algeria                                    1.83
              Angola                                     3.63
              Benin                                      3.00
              Botswana                                   1.96
              Burkina Faso                               3.02
...                                                       ...
South America Paraguay                                   1.60
              Peru                                       1.28
              Suriname                                   1.29
              Uruguay                                     .27
              Venezuela                                   .70

[188 rows x 1 columns]

## Query 2

How much more does the continent with strongest sports power spend on health compared to the continent with average sports power?

In [31]:
measures["healthcost"] = measures["health expenditure(GPT%).SUM"] * measures["GDP(pc).SUM"]

In [32]:
cube.query(measures["healthcost"], levels=[levels[("Region","Region", "region")], 
                                           levels[("Sport level","sportlevel","sportlevel")]])

healthcost
region        sportlevel                
Africa        Poor         31,442,313.40
Asia          Poor        123,507,133.43
Europe        Medium        2,159,869.97
              Poor        509,242,202.41
North America Poor         38,468,346.74
              Strong        1,195,343.76
Oceania       Poor         20,653,963.61
South America Poor          7,686,333.88

## Query 3

Do governments with middle health cost prefer to pay for their citizens' healthcare themselves, or do they let their citizens bear the costs?

In [33]:
cube.query(measures["healthdifference.SUM"], measures["GDP(pc).SUM"],levels=[levels[("Region","Region", "country")],levels[("HealthcostGPT","Healthcost Level","Healthcost Level")]])

healthdifference.SUM GDP(pc).SUM
region        country      Healthcost Level                                 
Africa        Algeria      Low                             54.17    3,354.16
              Angola       Low                             -5.94    1,502.95
              Benin        Low                             -4.13    1,237.95
              Botswana     Low                            199.78    5,875.07
              Burkina Faso Low                              2.75      833.24
...                                                          ...         ...
South America Paraguay     Medium                          35.18    5,353.35
              Peru         Low                            140.06    6,063.63
              Suriname     Low                            139.41    4,796.53
              Uruguay      Medium                         622.08   15,650.50
              Venezuela    Low                            -17.13         .00

[234 rows x 2 columns]

## Query 4
How does the GDP look like for middle countries with strong internet systems after hosting the Olympics?

In [34]:
cube.query( measures["GDP(pc).SUM"],levels=[levels[("Region","Region", "country")],
                                            levels[("hostcountry","hostbefore","hostbefore")],
                                            levels[("Internetpercountry","Internet Level","Internet Level")]])

GDP(pc).SUM
region        country      hostbefore Internet Level            
Africa        Algeria      none       Poor              3,354.16
              Angola       none       Poor              1,502.95
              Benin        none       Poor              1,237.95
              Botswana     none       Poor              5,875.07
              Burkina Faso none       Poor                833.24
...                                                          ...
South America Paraguay     none       Poor              5,353.35
              Peru         none       Poor              6,063.63
              Suriname     none       Poor              4,796.53
              Uruguay      none       Poor             15,650.50
              Venezuela    none       Poor                   .00

[234 rows x 1 columns]

## Query 5
the change in the number depress disorder in all continents that not good at sports, specifically before and after the 2002 Olympics.

In [35]:
cube.query(measures["depressive disorders rate of Change.SUM"], levels=[levels[("Region","Region", "region")],levels[("Sport level","sportlevel","sportlevel")]])

depressive disorders rate of Change.SUM
region        sportlevel                                        
Africa        Poor                                         68.74
Asia          Poor                                         -1.90
Europe        Medium                                        6.99
              Poor                                        -78.54
North America Poor                                         -2.60
              Strong                                        7.42
Oceania       Poor                                          9.15
South America Poor                                         17.20